<a href="https://colab.research.google.com/github/varun-k1601/kautilya-OA-SemanticSearch/blob/main/kautilya_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/xdevplatform/postman-twitter-api


Cloning into 'postman-twitter-api'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 65 (delta 9), reused 0 (delta 0), pack-reused 53 (from 1)
Receiving objects: 100% (65/65), 125.58 KiB | 7.85 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
!pip install sentence-transformers faiss-cpu



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 24.8 MB/s eta 0:00:00


In [ ]:
%%writefile semantic_search.py
import os
import json
import argparse
from typing import List, Dict, Any

import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# The repo you cloned lives at /content/postman-twitter-api in Colab
COLLECTION_PATH = os.path.join("postman-twitter-api", "Twitter API v2.postman_collection.json")


def load_collection(path: str = COLLECTION_PATH) -> List[Dict[str, Any]]:
    """Parse the Postman collection and extract meaningful text chunks."""
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    chunks: List[Dict[str, Any]] = []

    def walk_items(items, path_prefix: List[str]):
        for it in items:
            name = (it.get("name") or "").strip()
            new_path = path_prefix + ([name] if name else [])

            # Recurse into folders
            if "item" in it:
                walk_items(it["item"], new_path)

            # Extract request-level info (actual endpoints)
            req = it.get("request")
            if isinstance(req, dict):
                method = (req.get("method") or "").strip()

                url_obj = req.get("url")
                url = ""
                if isinstance(url_obj, dict):
                    url = (url_obj.get("raw") or "").strip()
                elif url_obj is not None:
                    url = str(url_obj).strip()

                desc = (req.get("description")
                        or it.get("description")
                        or "").strip()

                text_parts = [
                    " > ".join(new_path),
                    f"METHOD: {method}" if method else "",
                    f"URL: {url}" if url else "",
                    f"DESCRIPTION: {desc}" if desc else "",
                ]
                text = "\n".join([t for t in text_parts if t])

                if text.strip():
                    chunks.append(
                        {
                            "path": new_path,
                            "name": name,
                            "method": method,
                            "url": url,
                            "description": desc,
                            "text": text,
                        }
                    )

    items = data.get("item", [])
    walk_items(items, [])
    return chunks


def build_index(chunks: List[Dict[str, Any]],
                model_name: str = "all-MiniLM-L6-v2"):
    """Embed chunks and build a FAISS index."""
    if not chunks:
        raise ValueError("No chunks were parsed from the collection. Check COLLECTION_PATH.")

    texts = [c["text"] for c in chunks]

    model = SentenceTransformer(model_name)
    embeddings = model.encode(texts, show_progress_bar=True)
    embeddings = np.array(embeddings).astype("float32")

    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    return model, index, embeddings


def search(model,
           index: faiss.IndexFlatL2,
           chunks: List[Dict[str, Any]],
           query: str,
           top_k: int = 5):
    """Run semantic search and return top-k results."""
    if not chunks:
        return []

    top_k = min(top_k, len(chunks))

    query_emb = model.encode([query]).astype("float32")
    D, I = index.search(query_emb, top_k)

    results = []
    for rank, (score, idx) in enumerate(zip(D[0], I[0]), start=1):
        idx = int(idx)
        chunk = chunks[idx]
        results.append(
            {
                "rank": rank,
                "score": float(score),
                "chunk_id": idx,
                "name": chunk["name"],
                "method": chunk["method"],
                "url": chunk["url"],
                "path": " > ".join(chunk["path"]),
                "text": chunk["text"],
            }
        )
    return results


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--query", type=str, required=True)
    parser.add_argument("--top_k", type=int, default=5)
    args = parser.parse_args()

    chunks = load_collection(COLLECTION_PATH)
    if not chunks:
        print(json.dumps({"error": "No chunks parsed from collection"}, indent=2))
        return

    model, index, _ = build_index(chunks)
    results = search(model, index, chunks, args.query, top_k=args.top_k)

    print(json.dumps(results, indent=2))


if __name__ == "__main__":
    main()


Overwriting semantic_search.py


In [ ]:
!python semantic_search.py --query "How do I fetch tweets with expansions?"


2025-11-17 06:11:44.995076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763359905.019588    3376 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763359905.026741    3376 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763359905.045127    3376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763359905.045173    3376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763359905.045177    3376 computation_placer.cc:177] computation placer alr